# Control Flow

Amaranth提供了If/Elif/Else， Switch/Case 和 FMS/State控制流语法。使用with块进行上下文管理。

In [1]:
%config InteractiveShell.ast_node_interactivity = "all"

from amaranth import *

m = Module()
timer = Signal(8)

with m.If(timer == 0):
    m.d.sync += timer.eq(10)
with m.Else():
    m.d.sync += timer.eq(timer - 1)


上面的代码等效于：

In [2]:
timer = Signal(8)
m.d.sync += timer.eq(Mux(timer == 0, 10, timer - 1))

在Amaranth控制结构中的所有python代码始终按照它在程序中出现的顺序执行。可通过Python的print查看

In [3]:
timer = Signal(8)
with m.If(timer == 0):
    print("inside `If`")
    m.d.sync += timer.eq(10)
with m.Else():
    print("inside `Else`")
    m.d.sync += timer.eq(timer - 1)

inside `If`
inside `Else`


## Active and inactive assignments

在Amaranth控制结构中，即使用m.<...>块，如果满足控制条件，则为active状态，否则为inactive状态。

对于任何给定的条件集，模块中分配的每个信号的最终值与删除inactive assign

总之，一句话，m.<....>一定要写全，比如有If，必有Else。 有Switch， 必有default。

例如，一下代码生成circuit中有两种可能情况：

In [4]:
timer = Signal(8)
m.d.sync += timer.eq(timer - 1)
with m.If(timer == 0):
    m.d.sync += timer.eq(10)

当timer == 0时，代码简化为：

In [5]:
m.d.sync += timer.eq(timer - 1)
m.d.sync += timer.eq(10)

由于分配顺序，它进一步简化为：

In [6]:
m.d.sync += timer.eq(10)

当timer == 0 为false时，代码简化为

In [7]:
m.d.sync += timer.eq(timer - 1)

将这些情况组合在一起，上面的代码等小于：

In [8]:
timer = Signal(8)
m.d.sync += timer.eq(Mux(timer == 0, 10, timer - 1))

## If/Elif/Else控制块

条件控制流使用with m.If(cond1): 块来描述，该块后面可以跟一个或多个with m.Elif(cond2): 块，以及一个可选的final with m.Else():块。这种结构类似于Python的if/elif/else控制流语法

In [ ]:
with m.If(x_coord < 4):
    m.d.comb += is_bporch.eq(1)
    m.d.sync += x_coord.eq(x_coord + 1)
with m.Elif((x_coord >= 4) & (x_coord < 354)):
    m.d.comb += is_active.eq(1)
    m.d.sync += x_coord.eq(x_coord + 1)
with m.Elif((x_coord >= 364) & (x_coord < 374)):
    m.d.comb += is_fporch.eq(1)
    m.d.sync += x_coord.eq(x_coord + 1)
with m.Else():
    m.d.sync += x_coord.eq(0)

在单个If/Elif/Else块序列中，同一时刻只允许一个块处于active状态。

如果存在Else块，则一个块中的语句随时处于active状态，并且整个序列称为完整条件。

# Switch/Case控制块

根据不同的模式检查单个值，使用with m.switch(value):块来描述。

次块可以包含任意数量的witch m.Case(*patterns)和一个with m.Default(): 块，此结构与python的match/case控制流语法类似。

In [ ]:
value = Signal(4)

with m.Switch(value):
    with m.Case(0, 2, 4):
        m.d.comb += is_even.eq(1)
    with m.Case(1, 3, 5):
        m.d.comb += is_odd.eq(1)
    with m.Default():
        m.d.comb += too_big.eq(1)

在单个Switch块中，最多一个块的语句处于active状态。

这将是pattern与匹配的地一个Case块，或者地一个Default块，已较早者为准。

如果存在Default块，或者Case块中的模式涵盖了所有Switch值，则恰好一个块的语句将随时处于active状态，并且整个序列称为完整
条件。

虽然所有Amaranth控制流语法都可以通过编程方式生成，但Switch控制块特别容易以这种方式使用。

In [ ]:
length = Signal(4)
squared = Signal.like(length * length)

with m.Switch(length):
    for value in range(length.shape().width):
        with m.Case(value):
            m.d.comb += squared.eq(value * value)

# FSM/State控制块

简单的有限状态机使用with m.FSM(): 块来描述。

此块可以包含一个或多个with m.State("Name"): 块。

除了这些块之外，m.next = "Name" 语法还选择FSM在下一个clock cycle上进入的状态。

例如，此FSM在reset后执行一次总线读取事务。

In [ ]:
bus_addr = Signal(16)
r_data = Signal(8)
r_en = Signal()
latched = Signal.like(r_data)

with m.FSM():
    with m.State("Set Address"):
        m.d.sync += addr.eq(0x1234)
        m.next = "Strobe Read Enable"
    with m.State("Strobe Read Enable"):
        m.d.comb += r_en.eq(1)
        m.next = "Sample Data"
    with m.State("Sample Data"):
        m.d.sync += latched.eq(r_data)
        with m.If(r_data == 0):
            m.next = "Set Address" # try again

使用with m.FSM(init="Set Address"): 参数定义FSM时，可以提供FSM的初始状态。

如果未提供，则它是定义顺序的地一个状态。例如，此定义等小于本节开头的：

In [ ]:
with m.FSM(init="Set Address"):
    ....

FSM属于clock domain，该domain是使用with.FSM(domain="dom") 参数制定的。如果未指定，则为sync domain。

例如，此定义等效于本节开头的：

In [ ]:
with m.FSM(domain="sync"):
    ....

要确定（从FSM定义之外的代码中）它当前是否处于特定状态，可以捕获FSM

使用.ongoing("Name")方法返回一个值，该值在FSM处于相应状态时为true。

In [ ]:
with m.FSM() as fsm:
    ....

with m.If(fsm.ongoing("Set Address")):
    ...

请注意，在Python中，使用 with x() as y: 语法在块末尾之后仍然存在。

如果您在提供给m.next = ...或 fsm.ongoing(...)时，创建一个空名称且无法访问的状态，并且没有诊断信息。

这种错误的做法将来会修复。

如果使用m.State(...): 将非字符串对象作为状态名称，则首先将其强制转换成字符串，这可能会导致意外行为。

with m.Sate(...): 不专门处理枚举值，如果提供了URL，则将其转换为String，并且其数值将与生成的State Signal的数值没有对应关系。

这种错误的做法将来会修复

如果两个状态机相互嵌套，则m.next = ...句法始终引用最内层.要从内部状态及中更改外部状态机的状态，请使用中间信号。